In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard 

In [2]:
mp_hands = mp.solutions.hands # модель mediapipe hands для обнаружения ключевых точек
hands = mp_hands.Hands( 
    static_image_mode=False, 
    model_complexity=1, 
    min_detection_confidence=0.75, 
    min_tracking_confidence=0.75, 
    max_num_hands=2)

mp_drawing = mp.solutions.drawing_utils # утилита для отрисовки ключевых точек

# static_image_mode: It is used to specify whether the input image must be static images or as a video stream. The default value is False.
# model_complexity: Complexity of the hand landmark model: 0 or 1. Landmark accuracy, as well as inference latency, generally go up with the model complexity. Default to 1.
# min_detection_confidence: It is used to specify the minimum confidence value with which the detection from the person-detection model needs to be considered as successful. Can specify a value in [0.0,1.0]. The default value is 0.5.
# min_tracking_confidence: It is used to specify the minimum confidence value with which the detection from the landmark-tracking model must be considered as successful. Can specify a value in [0.0,1.0]. The default value is 0.5.
# max_num_hands: Maximum number of hands to detect. Default it is 2.

In [3]:
# получение значений из results 
def extract_keypoints(results):
    # в results.multi_hand_landmarks первые ключевые точки это правая рука, а вторые - левая
    right_hand_landmarks = None
    left_hand_landmarks = None

    if results.multi_handedness != None:
        if len(results.multi_handedness) == 2:
            right_hand_landmarks = results.multi_hand_landmarks[0]
            left_hand_landmarks = results.multi_hand_landmarks[1]
        else:
            if results.multi_handedness[0].classification[0].index == 1: #index right hand
                right_hand_landmarks = results.multi_hand_landmarks[0]
            else:
                left_hand_landmarks = results.multi_hand_landmarks[0]
            

    # 21 ориентир и 3 координаты у каждого x, y, z
    right_hand_np = np.array([[res.x, res.y, res.z] for res in right_hand_landmarks.landmark]).flatten() if right_hand_landmarks != None else np.zeros(21 * 3)
    left_hand_np = np.array([[res.x, res.y, res.z] for res in left_hand_landmarks.landmark]).flatten() if left_hand_landmarks != None else np.zeros(21 * 3)

    return np.concatenate([right_hand_np, left_hand_np]) # содержит ключевые точки элементов, которые представляют сглаженный массив значений x, y, z

In [4]:
def hand_detection(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    return image, results

In [9]:
# Тест
cap = cv2.VideoCapture(0)

while cap.isOpened():
        ret, frame = cap.read()

        image, results = hand_detection(frame)

        # Rendering Results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                # to change the colors you use this mp.drawing
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                          mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=4), )
                
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

print(extract_keypoints(results))
cap.release()
cv2.destroyAllWindows()

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

In [5]:
DATA_PATH = './data'
NP_DATA_PATH = './np_data'

# создание папки для датасета
try:
    os.makedirs(NP_DATA_PATH)
except FileExistsError:
    pass

for root, dirs, files in os.walk(DATA_PATH):

    for dir in dirs:
        try:
            os.makedirs(os.path.join(NP_DATA_PATH, dir))
        except FileExistsError:
            pass 
    
    file_count = 0
    for file in files:
        file_path = os.path.join(root, file)
            
        # Загружаем изображение с помощью OpenCV
        image = cv2.imread(file_path)
        image, results = hand_detection(image)

        # Export keypoints
        keypoints = extract_keypoints(results)
        # Проверка что массив не нулевой
        if not np.array_equal(keypoints, np.zeros(21 * 6)):
            npy_path = os.path.join(NP_DATA_PATH, os.path.basename(root), str(file_count))
            np.save(npy_path, keypoints)
            file_count += 1

        print(f'Обработан файл {file} из директории {os.path.basename(root)}')

Обработан файл 0.jpg из директории 0
Обработан файл 102.jpg из директории 0
Обработан файл 105.jpg из директории 0
Обработан файл 108.jpg из директории 0
Обработан файл 111.jpg из директории 0
Обработан файл 114.jpg из директории 0
Обработан файл 117.jpg из директории 0
Обработан файл 12.jpg из директории 0
Обработан файл 120.jpg из директории 0
Обработан файл 123.jpg из директории 0
Обработан файл 126.jpg из директории 0
Обработан файл 129.jpg из директории 0
Обработан файл 132.jpg из директории 0
Обработан файл 135.jpg из директории 0
Обработан файл 138.jpg из директории 0
Обработан файл 141.jpg из директории 0
Обработан файл 144.jpg из директории 0
Обработан файл 147.jpg из директории 0
Обработан файл 15.jpg из директории 0
Обработан файл 150.jpg из директории 0
Обработан файл 153.jpg из директории 0
Обработан файл 156.jpg из директории 0
Обработан файл 159.jpg из директории 0
Обработан файл 162.jpg из директории 0
Обработан файл 165.jpg из директории 0
Обработан файл 168.jpg из дир

In [32]:
X_data = []
y_data = []

for root, dirs, files in os.walk(NP_DATA_PATH):
    for file in files:
        np_arr = np.load(os.path.join(root, file))
        X_data.append(np_arr)
        y_data.append(int(os.path.basename(root)))

In [33]:
print(y_data[0])
print(len(X_data))

y_data = np.array(y_data)
X_data = np.array(X_data)

0
11774


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.10, random_state=99)

In [35]:
len(set(y_data))

68

In [78]:
NUM_CLASSES = 68

model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 6, )),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [79]:
model.summary() 

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 256)               32512     
                                                                 
 dropout_15 (Dropout)        (None, 256)               0         
                                                                 
 dense_34 (Dense)            (None, 128)               32896     
                                                                 
 dense_35 (Dense)            (None, 68)                8772      
                                                                 
Total params: 74,180
Trainable params: 74,180
Non-trainable params: 0
_________________________________________________________________


In [80]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [81]:
log_dir = os.path.join('mediapipe_hands', 'V14')
tb_callback = TensorBoard(log_dir=log_dir)

In [82]:
# обучение модели
model.fit(X_train, y_train, epochs=1000, batch_size=64, callbacks=[tb_callback])

Epoch 1/1000
166/166 [==============================] - 0s 1ms/step - loss: 4.0975 - accuracy: 0.0455
Epoch 2/1000
166/166 [==============================] - 0s 1ms/step - loss: 3.9263 - accuracy: 0.0699
Epoch 3/1000
166/166 [==============================] - 0s 1ms/step - loss: 3.7884 - accuracy: 0.0820
Epoch 4/1000
166/166 [==============================] - 0s 1ms/step - loss: 3.6620 - accuracy: 0.1042
Epoch 5/1000
166/166 [==============================] - 0s 1ms/step - loss: 3.5340 - accuracy: 0.1236
Epoch 6/1000
166/166 [==============================] - 0s 1ms/step - loss: 3.4047 - accuracy: 0.1412
Epoch 7/1000
166/166 [==============================] - 0s 1ms/step - loss: 3.2669 - accuracy: 0.1604
Epoch 8/1000
166/166 [==============================] - 0s 1ms/step - loss: 3.1438 - accuracy: 0.1737
Epoch 9/1000
166/166 [==============================] - 0s 1ms/step - loss: 3.0008 - accuracy: 0.2015
Epoch 10/1000
166/166 [==============================] - 0s 1ms/step - loss: 2.897

In [83]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=64)

19/19 [==============================] - 0s 650us/step - loss: 0.5994 - accuracy: 0.9066
